In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    
    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']   = (Src['Close'] - Src['Open'])
    Src['Hilo']     = (Src['High']  - Src['Low'])
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = (-1) * Calc.groupby('Date', sort=0)['Time'].rank(ascending=False)

    Calc[f'Change Pos'] = (Calc['Change'] >= 0)
    Calc[f'Change Neg'] = (Calc['Change'] <  0)

    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Change +{x}']     =  Calc.groupby('Date', sort=0)[f'Change'].shift(-x)
    for x in FUTS:  Calc[f'Change +{x} Pos'] = (Calc[f'Change +{x}'] >= 0)
    for x in FUTS:  Calc[f'Change +{x} Neg'] = (Calc[f'Change +{x}'] <  0)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [16]:
pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[(Calc['T'] < (-1)) & Calc['Change Pos']][['Change +1 Pos','Change +1 Neg']].mean().mul(100).round(1) }, 
    { 'Condition':'Change Neg', **Calc[(Calc['T'] < (-1)) & Calc['Change Neg']][['Change +1 Pos','Change +1 Neg']].mean().mul(100).round(1) }, 
])

,Condition,Change +1 Pos,Change +1 Neg
0,Change Pos,50.4,49.6
1,Change Neg,50.8,49.2


# Snippets

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change'] = (Src['Close'] - Src['Open'])
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = (-1) * Calc.groupby('Date', sort=0)['Time'].rank(ascending=False)

    Calc[f'Change Pos'] = (Calc['Change'] >= 0)
    Calc[f'Change Neg'] = (Calc['Change'] <  0)

    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Change +{x}']     =  Calc.groupby('Date', sort=0)[f'Change'].shift(-x)
    for x in FUTS:  Calc[f'Change +{x} Pos'] = (Calc[f'Change +{x}'] >= 0)
    for x in FUTS:  Calc[f'Change +{x} Neg'] = (Calc[f'Change +{x}'] <  0)
    return Calc


pd.DataFrame([
    { 'Condition':'Change Pos', **Calc[(Calc['T'] < (-1)) & Calc['Change Pos']][['Change +1 Pos','Change +1 Neg']].mean() }, 
    { 'Condition':'Change Neg', **Calc[(Calc['T'] < (-1)) & Calc['Change Neg']][['Change +1 Pos','Change +1 Neg']].mean() }, 
])